In [1]:
from IPython.core.debugger import Tracer
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models
import argparse
from datetime import datetime
import os
import time
import multiprocessing
import psutil
import json
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd  
import h5py
from sklearn.model_selection import train_test_split
from skimage.util import crop,  random_noise
from skimage.transform import   rescale, resize, rotate, AffineTransform, warp
import torch.optim as optim
from tqdm import tqdm
from resnet18 import resnet18
from collections import Counter
from util import get_statistics
from dataset import Dataset_Generator, train_validation_test_split, get_classes_map, number_of_classes, number_of_channels, get_all_object_numbers_labels
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import ast

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 300
n_epochs = 100
num_workers = 0
lr = 0.001
n_splits = 5

In [3]:
weights = [10.0,10.1,10.4,1.0,1.0,10.0,10.0,10.0,10.0]

In [3]:
#only_channels = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
only_channels = []
only_classes = None

In [4]:
h5_file = "data/WBC/Lyse fix sample_1_Focused & Singlets & CD45 pos.h5"

In [5]:
label_map = get_classes_map(h5_file)

In [6]:
class_names = list(label_map.values())

In [7]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1., p=0.5):
        self.std = std
        self.mean = mean
        self.p = p
        
    def __call__(self, tensor):
        if torch.rand(1) < self.p:
            return tensor + torch.randn(tensor.size()) * self.std + self.mean
        return tensor
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [8]:
transform = transforms.Compose(
        [transforms.RandomVerticalFlip(),
         transforms.RandomHorizontalFlip(),
         transforms.RandomRotation(45),
        AddGaussianNoise(0., 1., 0.3)])

In [9]:
num_classes = number_of_classes(h5_file, only_classes=only_classes)
num_channels = number_of_channels(h5_file, only_channels=only_channels)

In [10]:
label_map

{0: ' unknown',
 1: ' CD4+ T',
 2: ' CD8+ T',
 3: ' CD15+ neutrophil',
 4: ' CD14+ monocyte',
 5: ' CD19+ B',
 6: ' CD56+ NK',
 7: ' NKT',
 8: ' eosinophil'}

In [12]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#rus = RandomUnderSampler(random_state=42, sampling_strategy={0: 1285, 1: 4475, 2: 1787, 3:10000, 4:1281, 5:888, 6:668, 7:1027, 8:1536})
#rus = RandomUnderSampler(random_state=42, sampling_strategy={3:4476})
#rus = RandomUnderSampler(random_state=42, sampling_strategy='majority')
#rus = RandomUnderSampler(random_state=42, sampling_strategy={0: 1000, 1: 1000, 2: 1000, 3:1000, 4:1000, 5:888, 6:668, 7:1000, 8:1000})
#ros = RandomOverSampler(random_state=42, sampling_strategy='not majority')
ros = RandomOverSampler(random_state=42, sampling_strategy='all')
#rus = RandomUnderSampler(random_state=42, sampling_strategy='not minority')
#ros = RandomOverSampler(random_state=42, sampling_strategy={0: 1000, 1: 1000, 2: 1000, 3:1000, 4:1000, 5:1000, 6:1000, 7:1000, 8:1000})

In [13]:
class oversampled_Kfold():
    def __init__(self, n_splits, n_repeats=1):
        self.n_splits = n_splits
        self.n_repeats = n_repeats

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits*self.n_repeats

    def split(self, X, y, groups=None):
        splits = np.array_split(np.random.choice(len(X), len(X),replace=False), self.n_splits)
        train, test = [], []
        for repeat in range(self.n_repeats):
            for idx in range(len(splits)):
                trainingIdx = np.delete(splits, idx)
                Xidx_r, y_r = ros.fit_resample(np.hstack(trainingIdx).reshape((-1,1)), np.asarray(y[np.hstack(trainingIdx)]))
                #Tracer()()
                #Xidx_r, y_r = ros.fit_resample(Xidx_r, y_r)
                #Tracer()()
                train.append(Xidx_r.flatten())
                test.append(splits[idx])
        return list(zip(train, test))

In [14]:
rkf_search = oversampled_Kfold(n_splits=n_splits, n_repeats=1)

In [15]:
X, y = get_all_object_numbers_labels(h5_file, only_classes) 

In [16]:
best_accuracy = 0.0

In [20]:
print("Start validation")
for train_indx, test_indx in rkf_search.split(X, y):
    train_dataset = Dataset_Generator(h5_file, train_indx, reshape_size=64, transform=transform,
                                      only_channels=only_channels, only_classes=only_classes)
    trainloader = DataLoader(train_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=num_workers)
    statistics = get_statistics(trainloader, only_channels)
    print('statistics used: %s' % (str(statistics)))
    print('length of the dataloader is: %s' % (str(len(trainloader))))
    train_dataset = Dataset_Generator(h5_file, train_indx, reshape_size=64, transform=transform,
                                      means=statistics["mean"].div_(len(trainloader)),
                                      stds=statistics["std"].div_(len(trainloader)), only_channels=only_channels,
                                      only_classes=only_classes)
    test_dataset = Dataset_Generator(h5_file, test_indx, reshape_size=64,
                                     means=statistics["mean"].div_(len(trainloader)),
                                     stds=statistics["std"].div_(len(trainloader)), only_channels=only_channels,
                                     only_classes=only_classes)
    trainloader = DataLoader(train_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=num_workers)
    testloader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=num_workers)
    model = resnet18(pretrained=True)

    # loading the imagenet weights in case it is possible
    if num_channels != 3:
        model.conv1 = nn.Conv2d(num_channels, 64, kernel_size=(7, 7),
                                    stride=(2, 2), padding=(3, 3), bias=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    model = model.to(device)
    #class_weights = torch.FloatTensor(weights).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    for epoch in range(n_epochs):
        running_loss = 0.0
        print('epoch%d' % epoch)
        for i, data in enumerate(trainloader, 0):
            indx = (data["object_number"] != -1).reshape(-1)
            if indx.sum() > 0:
                inputs, labels = data["image"][indx], data["label"][indx]

                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.float()
                labels = labels.reshape(-1)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)

                loss = criterion(outputs, F.one_hot(labels.long(), num_classes).type_as(outputs))
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
            else:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            if i % 100 == 99:  # print every 2000 mini-batches
                print('[%d, %5d] training loss: %.8f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    correct = 0.
    total = 0.
    y_true = list()
    y_pred = list()

    with torch.no_grad():
        for data in testloader:
            indx = (data["object_number"] != -1).reshape(-1)
            if indx.sum() > 0:
                inputs, labels = data["image"][indx], data["label"][indx]

                inputs, labels = inputs.to(device), labels.to(device)     
                inputs = inputs.float()
                labels = labels.reshape(-1)

                outputs = model(inputs)
                pred = outputs.argmax(dim=1)
                _ , predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (labels.reshape(-1) == predicted).sum().item()
                for i in range(len(pred)):
                    y_true.append(labels[i].item())
                    y_pred.append(pred[i].item())

    print('Accuracy of the network on the %d test images: %d %%' % (len(test_dataset),
        100 * correct / total))
    if 100 * correct / total > best_accuracy:
        torch.save(model.state_dict(), os.path.join("models/final_model_dict_best_metrics.pth"))
        print('test_indx used: %s' % (', '.join(str(x) for x in np.unique(test_indx))))
        Tracer()()
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    f1_score_original = f1_score(y_true, y_pred, average=None, labels=np.arange(num_classes))
    df = pd.DataFrame(np.atleast_2d(f1_score_original), columns=class_names)
    print(df.to_string())
    torch.cuda.empty_cache()

Start validation
statistics used: {'mean': tensor([95.3586,  3.0924,  7.1274,  2.7574,  3.7047,  6.7795,  7.5772,  6.3744,
        93.2530,  6.6463,  2.8277,  4.3756]), 'std': tensor([303.5554, 301.5881, 301.5339, 301.5101, 301.5269, 301.7011, 301.6101,
        301.5654, 303.4497, 301.5149, 301.5242, 301.5264])}
statistics used: {'mean': tensor([95.3586,  3.0924,  7.1274,  2.7574,  3.7047,  6.7795,  7.5772,  6.3744,
        93.2530,  6.6463,  2.8277,  4.3756]), 'std': tensor([303.5554, 301.5881, 301.5339, 301.5101, 301.5269, 301.7011, 301.6101,
        301.5654, 303.4497, 301.5149, 301.5242, 301.5264])}
length of the dataloader is: 550
epoch0
[1,   100] training loss: 0.01154879
[1,   200] training loss: 0.01060274
[1,   300] training loss: 0.01355968
[1,   400] training loss: 0.01453920
[1,   500] training loss: 0.01135116
epoch1
[2,   100] training loss: 0.01123228
[2,   200] training loss: 0.01206790
[2,   300] training loss: 0.01458214
[2,   400] training loss: 0.01508419
[2,   500

<ipython-input-20-eb981bfcb3d5>:100: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  Tracer()()


Accuracy of the network on the 6256 test images: 96 %
test_indx used: 2, 5, 14, 17, 20, 21, 24, 25, 27, 38, 44, 49, 50, 63, 71, 74, 77, 82, 89, 90, 126, 129, 131, 133, 134, 138, 154, 155, 158, 173, 180, 181, 182, 185, 193, 196, 206, 215, 240, 242, 249, 252, 257, 265, 280, 287, 299, 303, 308, 325, 333, 339, 344, 354, 357, 368, 370, 379, 383, 387, 402, 412, 418, 420, 421, 423, 424, 434, 435, 441, 447, 450, 455, 463, 469, 471, 475, 483, 484, 490, 491, 511, 512, 514, 515, 521, 533, 536, 539, 542, 556, 557, 562, 571, 579, 592, 595, 598, 600, 611, 612, 616, 626, 629, 635, 649, 657, 658, 666, 668, 669, 675, 679, 689, 691, 696, 703, 709, 710, 716, 724, 732, 733, 734, 736, 740, 743, 745, 746, 750, 760, 770, 779, 793, 797, 801, 804, 805, 811, 814, 815, 816, 817, 823, 829, 833, 851, 856, 863, 867, 869, 872, 874, 886, 887, 894, 897, 908, 915, 916, 918, 919, 920, 921, 927, 940, 965, 975, 976, 980, 983, 987, 992, 993, 1004, 1006, 1012, 1024, 1032, 1040, 1045, 1047, 1053, 1056, 1059, 1060, 1064, 1065

ipdb>  classification_report(y_true, y_pred, target_names=class_names, digits=4)


'                   precision    recall  f1-score   support\n\n          unknown     0.8141    0.5226    0.6366       243\n           CD4+ T     0.9687    0.9968    0.9825       932\n           CD8+ T     0.9330    0.9305    0.9317       374\n CD15+ neutrophil     0.9986    1.0000    0.9993      3603\n   CD14+ monocyte     0.9692    0.9895    0.9792       286\n          CD19+ B     0.8424    0.9942    0.9120       172\n         CD56+ NK     0.8790    0.9650    0.9200       143\n              NKT     0.7308    0.7308    0.7308       208\n       eosinophil     0.9800    0.9966    0.9882       295\n\n         accuracy                         0.9663      6256\n        macro avg     0.9017    0.9029    0.8978      6256\n     weighted avg     0.9649    0.9663    0.9641      6256\n'


...

ipdb>  c


                   precision    recall  f1-score   support

          unknown     0.8141    0.5226    0.6366       243
           CD4+ T     0.9687    0.9968    0.9825       932
           CD8+ T     0.9330    0.9305    0.9317       374
 CD15+ neutrophil     0.9986    1.0000    0.9993      3603
   CD14+ monocyte     0.9692    0.9895    0.9792       286
          CD19+ B     0.8424    0.9942    0.9120       172
         CD56+ NK     0.8790    0.9650    0.9200       143
              NKT     0.7308    0.7308    0.7308       208
       eosinophil     0.9800    0.9966    0.9882       295

         accuracy                         0.9663      6256
        macro avg     0.9017    0.9029    0.8978      6256
     weighted avg     0.9649    0.9663    0.9641      6256

    unknown    CD4+ T    CD8+ T   CD15+ neutrophil   CD14+ monocyte   CD19+ B   CD56+ NK       NKT   eosinophil
0  0.636591  0.982549  0.931727           0.999307         0.979239     0.912       0.92  0.730769     0.988235

Progr

ipdb>  q


statistics used: {'mean': tensor([95.3458,  3.0811,  7.1079,  2.7316,  3.7187,  6.8049,  7.5921,  6.4025,
        93.2402,  6.6470,  2.8097,  4.3540]), 'std': tensor([302.3926, 300.4296, 300.3419, 300.3299, 300.3427, 300.5024, 300.4545,
        300.3837, 302.3014, 300.3617, 300.3475, 300.3578])}
statistics used: {'mean': tensor([95.3458,  3.0811,  7.1079,  2.7316,  3.7187,  6.8049,  7.5921,  6.4025,
        93.2402,  6.6470,  2.8097,  4.3540]), 'std': tensor([302.3926, 300.4296, 300.3419, 300.3299, 300.3427, 300.5024, 300.4545,
        300.3837, 302.3014, 300.3617, 300.3475, 300.3578])}
length of the dataloader is: 550
epoch0
[1,   100] training loss: 0.01165580
[1,   200] training loss: 0.01051884
[1,   300] training loss: 0.01356692
[1,   400] training loss: 0.01467885
[1,   500] training loss: 0.01142789
epoch1
[2,   100] training loss: 0.01125337
[2,   200] training loss: 0.01187742
[2,   300] training loss: 0.01452840
[2,   400] training loss: 0.01517787


KeyboardInterrupt: 

In [ ]:
 {'mean': tensor([95.3616,  6.5219,  7.1357,  2.5799,  3.6773,  7.7228,  7.2317,  5.5552,
        93.9901,  6.6885,  2.8189,  4.2796]), 'std': tensor([301.9253, 300.1979, 299.8447, 299.8529, 299.8598, 300.0155, 299.9408,
        299.8964, 301.8691, 299.8426, 299.8632, 299.8602])}

In [11]:
statistics = {'mean': torch.tensor([95.3458,  3.0811,  7.1079,  2.7316,  3.7187,  6.8049,  7.5921,  6.4025,
        93.2402,  6.6470,  2.8097,  4.3540]), 'std': torch.tensor([302.3926, 300.4296, 300.3419, 300.3299, 300.3427, 300.5024, 300.4545,
        300.3837, 302.3014, 300.3617, 300.3475, 300.3578])}

In [12]:
model = resnet18(pretrained=True)
if num_channels != 3:
    model.conv1 = nn.Conv2d(num_channels, 64, kernel_size=(7, 7),
                                stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)
model.load_state_dict(torch.load("models/final_model_dict_best_metrics_h5_local.pth", map_location=torch.device(device)))

<All keys matched successfully>

In [13]:
test_indx = [2, 5, 14, 17, 20, 21, 24, 25, 27, 38, 44, 49, 50, 63, 71, 74, 77, 82, 89, 90, 126, 129, 131, 133, 134, 138, 154, 155, 158, 173, 180, 181, 182, 185, 193, 196, 206, 215, 240, 242, 249, 252, 257, 265, 280, 287, 299, 303, 308, 325, 333, 339, 344, 354, 357, 368, 370, 379, 383, 387, 402, 412, 418, 420, 421, 423, 424, 434, 435, 441, 447, 450, 455, 463, 469, 471, 475, 483, 484, 490, 491, 511, 512, 514, 515, 521, 533, 536, 539, 542, 556, 557, 562, 571, 579, 592, 595, 598, 600, 611, 612, 616, 626, 629, 635, 649, 657, 658, 666, 668, 669, 675, 679, 689, 691, 696, 703, 709, 710, 716, 724, 732, 733, 734, 736, 740, 743, 745, 746, 750, 760, 770, 779, 793, 797, 801, 804, 805, 811, 814, 815, 816, 817, 823, 829, 833, 851, 856, 863, 867, 869, 872, 874, 886, 887, 894, 897, 908, 915, 916, 918, 919, 920, 921, 927, 940, 965, 975, 976, 980, 983, 987, 992, 993, 1004, 1006, 1012, 1024, 1032, 1040, 1045, 1047, 1053, 1056, 1059, 1060, 1064, 1065, 1074, 1075, 1086, 1088, 1090, 1094, 1096, 1105, 1109, 1111, 1119, 1129, 1132, 1137, 1139, 1152, 1163, 1166, 1173, 1183, 1192, 1195, 1199, 1209, 1211, 1213, 1217, 1224, 1225, 1229, 1235, 1239, 1242, 1249, 1250, 1257, 1258, 1259, 1260, 1265, 1268, 1274, 1275, 1277, 1280, 1281, 1284, 1285, 1289, 1291, 1293, 1294, 1296, 1300, 1302, 1305, 1307, 1312, 1313, 1317, 1320, 1327, 1328, 1334, 1338, 1344, 1348, 1362, 1364, 1372, 1374, 1382, 1386, 1394, 1397, 1405, 1409, 1410, 1416, 1419, 1427, 1430, 1432, 1434, 1438, 1444, 1449, 1451, 1460, 1467, 1469, 1474, 1477, 1478, 1482, 1491, 1492, 1493, 1497, 1499, 1500, 1501, 1503, 1504, 1508, 1512, 1536, 1537, 1545, 1547, 1554, 1566, 1568, 1576, 1581, 1584, 1586, 1592, 1597, 1605, 1609, 1612, 1620, 1621, 1629, 1632, 1646, 1652, 1654, 1660, 1662, 1666, 1676, 1681, 1682, 1688, 1691, 1700, 1706, 1720, 1721, 1728, 1736, 1737, 1752, 1761, 1762, 1782, 1787, 1790, 1793, 1796, 1801, 1804, 1805, 1812, 1813, 1825, 1826, 1831, 1833, 1834, 1847, 1849, 1856, 1858, 1859, 1863, 1865, 1872, 1878, 1891, 1902, 1905, 1910, 1917, 1921, 1925, 1926, 1929, 1930, 1932, 1935, 1942, 1943, 1952, 1954, 1955, 1958, 1967, 1973, 1985, 1993, 1994, 1995, 1996, 1999, 2000, 2002, 2014, 2021, 2035, 2042, 2052, 2060, 2061, 2063, 2066, 2078, 2079, 2081, 2086, 2088, 2092, 2098, 2100, 2101, 2108, 2113, 2119, 2121, 2122, 2130, 2131, 2139, 2140, 2141, 2143, 2144, 2147, 2148, 2153, 2156, 2164, 2165, 2167, 2169, 2172, 2173, 2184, 2187, 2190, 2195, 2196, 2199, 2200, 2209, 2216, 2217, 2218, 2219, 2220, 2226, 2228, 2234, 2244, 2248, 2249, 2256, 2266, 2267, 2272, 2274, 2278, 2281, 2283, 2284, 2286, 2287, 2292, 2311, 2312, 2317, 2322, 2328, 2331, 2345, 2346, 2356, 2358, 2367, 2369, 2371, 2378, 2381, 2390, 2393, 2397, 2406, 2409, 2411, 2412, 2419, 2422, 2432, 2434, 2436, 2445, 2447, 2453, 2468, 2469, 2472, 2476, 2478, 2488, 2499, 2501, 2502, 2505, 2509, 2510, 2513, 2515, 2530, 2539, 2541, 2542, 2543, 2547, 2551, 2557, 2559, 2560, 2563, 2566, 2568, 2571, 2577, 2594, 2597, 2598, 2602, 2607, 2610, 2613, 2614, 2622, 2626, 2633, 2634, 2639, 2644, 2645, 2653, 2661, 2667, 2668, 2671, 2675, 2677, 2686, 2688, 2697, 2700, 2705, 2724, 2731, 2739, 2741, 2750, 2754, 2760, 2768, 2771, 2773, 2777, 2785, 2788, 2789, 2790, 2792, 2797, 2801, 2808, 2817, 2824, 2825, 2827, 2838, 2844, 2855, 2865, 2867, 2872, 2876, 2879, 2880, 2882, 2885, 2888, 2892, 2900, 2902, 2911, 2913, 2924, 2930, 2932, 2949, 2961, 2962, 2966, 2977, 2981, 2985, 2988, 2991, 2993, 2995, 3013, 3023, 3029, 3031, 3033, 3037, 3039, 3042, 3043, 3044, 3047, 3048, 3055, 3065, 3071, 3075, 3076, 3086, 3090, 3103, 3106, 3107, 3108, 3110, 3115, 3116, 3118, 3119, 3120, 3134, 3145, 3153, 3155, 3156, 3160, 3161, 3166, 3174, 3183, 3184, 3188, 3204, 3209, 3212, 3222, 3223, 3227, 3233, 3238, 3243, 3245, 3249, 3255, 3256, 3261, 3262, 3263, 3265, 3290, 3293, 3294, 3296, 3297, 3298, 3303, 3314, 3316, 3319, 3324, 3330, 3332, 3342, 3343, 3345, 3356, 3359, 3361, 3368, 3378, 3379, 3380, 3382, 3388, 3389, 3394, 3396, 3400, 3403, 3417, 3419, 3424, 3427, 3433, 3435, 3436, 3437, 3442, 3444, 3452, 3456, 3460, 3465, 3469, 3473, 3476, 3477, 3486, 3487, 3495, 3496, 3498, 3501, 3513, 3515, 3517, 3518, 3519, 3520, 3523, 3528, 3529, 3531, 3534, 3536, 3541, 3549, 3552, 3560, 3564, 3565, 3572, 3573, 3575, 3579, 3584, 3588, 3591, 3595, 3596, 3602, 3604, 3619, 3624, 3629, 3635, 3638, 3643, 3651, 3653, 3654, 3655, 3659, 3660, 3661, 3665, 3667, 3679, 3680, 3683, 3693, 3695, 3701, 3702, 3705, 3708, 3711, 3712, 3714, 3727, 3729, 3741, 3744, 3750, 3756, 3766, 3769, 3771, 3775, 3781, 3790, 3794, 3798, 3805, 3807, 3808, 3819, 3820, 3822, 3827, 3832, 3840, 3850, 3853, 3858, 3867, 3877, 3880, 3882, 3885, 3886, 3896, 3901, 3906, 3907, 3912, 3921, 3923, 3926, 3929, 3930, 3932, 3934, 3938, 3946, 3947, 3948, 3952, 3960, 3963, 3968, 3969, 3973, 3978, 3982, 3985, 3987, 3988, 3994, 4001, 4003, 4007, 4008, 4012, 4016, 4033, 4037, 4042, 4054, 4056, 4062, 4066, 4069, 4074, 4085, 4088, 4105, 4113, 4119, 4123, 4125, 4127, 4136, 4149, 4151, 4153, 4157, 4163, 4170, 4173, 4180, 4182, 4184, 4188, 4190, 4201, 4207, 4209, 4217, 4223, 4231, 4233, 4237, 4241, 4250, 4253, 4255, 4269, 4270, 4274, 4289, 4294, 4296, 4302, 4303, 4308, 4309, 4314, 4320, 4327, 4328, 4340, 4341, 4354, 4366, 4370, 4371, 4387, 4388, 4396, 4406, 4414, 4418, 4419, 4445, 4446, 4458, 4466, 4467, 4469, 4477, 4486, 4488, 4496, 4502, 4507, 4515, 4518, 4528, 4529, 4544, 4548, 4554, 4555, 4557, 4560, 4563, 4566, 4570, 4572, 4574, 4575, 4578, 4584, 4588, 4609, 4614, 4623, 4627, 4635, 4644, 4648, 4651, 4653, 4655, 4662, 4664, 4666, 4669, 4672, 4678, 4686, 4690, 4691, 4700, 4702, 4705, 4706, 4716, 4718, 4719, 4721, 4727, 4728, 4747, 4752, 4761, 4762, 4766, 4768, 4771, 4779, 4784, 4791, 4794, 4799, 4808, 4810, 4812, 4814, 4823, 4829, 4831, 4832, 4836, 4838, 4841, 4846, 4847, 4848, 4851, 4853, 4864, 4868, 4869, 4873, 4877, 4879, 4882, 4884, 4886, 4888, 4893, 4904, 4910, 4913, 4917, 4919, 4922, 4924, 4935, 4937, 4941, 4943, 4949, 4958, 4961, 4964, 4970, 4984, 4998, 5009, 5019, 5021, 5023, 5027, 5028, 5029, 5036, 5055, 5056, 5059, 5061, 5062, 5068, 5074, 5075, 5079, 5083, 5091, 5106, 5112, 5113, 5123, 5126, 5132, 5133, 5138, 5151, 5154, 5161, 5166, 5167, 5170, 5179, 5182, 5192, 5195, 5197, 5201, 5207, 5218, 5224, 5231, 5234, 5240, 5245, 5250, 5258, 5260, 5262, 5268, 5272, 5279, 5281, 5284, 5296, 5299, 5302, 5308, 5311, 5313, 5315, 5318, 5319, 5321, 5323, 5328, 5329, 5333, 5341, 5346, 5356, 5359, 5362, 5364, 5375, 5385, 5395, 5396, 5408, 5409, 5412, 5417, 5420, 5425, 5428, 5430, 5433, 5437, 5438, 5439, 5440, 5441, 5443, 5449, 5456, 5458, 5463, 5470, 5485, 5492, 5508, 5524, 5526, 5527, 5528, 5537, 5543, 5551, 5558, 5560, 5575, 5577, 5587, 5588, 5589, 5598, 5599, 5604, 5606, 5607, 5608, 5621, 5624, 5627, 5628, 5629, 5631, 5634, 5637, 5642, 5644, 5666, 5678, 5679, 5693, 5694, 5710, 5712, 5713, 5714, 5721, 5724, 5726, 5733, 5737, 5739, 5740, 5745, 5748, 5749, 5750, 5756, 5761, 5762, 5764, 5768, 5771, 5774, 5778, 5780, 5783, 5798, 5799, 5803, 5807, 5818, 5821, 5824, 5839, 5842, 5854, 5862, 5870, 5883, 5889, 5894, 5899, 5904, 5908, 5928, 5929, 5931, 5932, 5935, 5943, 5958, 5961, 5963, 5966, 5968, 5980, 5984, 5989, 5990, 6008, 6009, 6014, 6017, 6029, 6030, 6032, 6034, 6041, 6043, 6051, 6056, 6077, 6085, 6089, 6096, 6104, 6106, 6108, 6111, 6115, 6117, 6123, 6125, 6127, 6130, 6131, 6146, 6147, 6159, 6191, 6193, 6195, 6203, 6204, 6208, 6210, 6215, 6217, 6220, 6225, 6233, 6241, 6242, 6248, 6249, 6252, 6257, 6259, 6270, 6275, 6283, 6287, 6293, 6301, 6302, 6305, 6311, 6313, 6314, 6317, 6325, 6329, 6330, 6332, 6333, 6335, 6348, 6351, 6353, 6355, 6356, 6357, 6362, 6365, 6375, 6376, 6379, 6380, 6385, 6389, 6403, 6405, 6411, 6434, 6437, 6439, 6440, 6450, 6451, 6455, 6457, 6459, 6464, 6469, 6471, 6477, 6481, 6482, 6484, 6486, 6489, 6492, 6496, 6497, 6501, 6506, 6507, 6508, 6509, 6511, 6520, 6522, 6526, 6537, 6538, 6545, 6546, 6549, 6553, 6558, 6564, 6572, 6583, 6585, 6600, 6607, 6608, 6611, 6614, 6641, 6643, 6651, 6653, 6670, 6686, 6691, 6692, 6699, 6700, 6701, 6705, 6711, 6713, 6730, 6732, 6738, 6743, 6744, 6752, 6754, 6759, 6770, 6771, 6773, 6778, 6781, 6788, 6791, 6792, 6802, 6804, 6810, 6818, 6841, 6846, 6848, 6850, 6869, 6871, 6875, 6880, 6881, 6883, 6886, 6893, 6894, 6896, 6902, 6903, 6905, 6921, 6930, 6941, 6952, 6964, 6966, 6971, 6973, 6979, 6990, 6996, 6999, 7008, 7020, 7021, 7030, 7031, 7033, 7039, 7044, 7047, 7051, 7053, 7056, 7063, 7075, 7082, 7084, 7086, 7091, 7093, 7097, 7099, 7118, 7122, 7130, 7134, 7142, 7143, 7144, 7147, 7148, 7160, 7166, 7168, 7171, 7173, 7182, 7188, 7190, 7194, 7196, 7201, 7205, 7217, 7222, 7231, 7237, 7241, 7243, 7247, 7254, 7272, 7279, 7284, 7287, 7289, 7291, 7308, 7311, 7317, 7325, 7326, 7338, 7340, 7344, 7346, 7348, 7350, 7367, 7370, 7374, 7393, 7396, 7401, 7410, 7411, 7414, 7422, 7424, 7430, 7433, 7436, 7439, 7441, 7447, 7467, 7469, 7471, 7472, 7476, 7484, 7493, 7497, 7499, 7505, 7507, 7521, 7525, 7532, 7536, 7540, 7545, 7547, 7549, 7550, 7551, 7559, 7563, 7564, 7566, 7585, 7586, 7591, 7596, 7609, 7615, 7616, 7620, 7629, 7632, 7634, 7635, 7644, 7653, 7656, 7664, 7668, 7669, 7681, 7682, 7689, 7690, 7692, 7700, 7703, 7708, 7709, 7712, 7719, 7728, 7730, 7733, 7737, 7740, 7741, 7748, 7753, 7759, 7763, 7767, 7769, 7774, 7778, 7787, 7789, 7790, 7791, 7800, 7801, 7808, 7810, 7817, 7818, 7826, 7834, 7850, 7853, 7855, 7863, 7864, 7872, 7876, 7881, 7884, 7899, 7900, 7904, 7913, 7920, 7926, 7929, 7955, 7958, 7959, 7963, 7967, 7969, 7973, 7975, 7978, 7981, 7982, 7984, 7986, 7988, 7992, 8000, 8010, 8012, 8013, 8014, 8020, 8021, 8023, 8026, 8028, 8032, 8037, 8048, 8054, 8059, 8060, 8070, 8078, 8084, 8099, 8102, 8104, 8107, 8112, 8115, 8127, 8130, 8131, 8132, 8137, 8140, 8143, 8148, 8149, 8151, 8157, 8166, 8173, 8177, 8180, 8188, 8191, 8193, 8196, 8201, 8211, 8214, 8215, 8217, 8223, 8226, 8238, 8242, 8244, 8250, 8251, 8252, 8258, 8263, 8265, 8268, 8282, 8285, 8286, 8288, 8292, 8293, 8322, 8324, 8328, 8337, 8338, 8339, 8342, 8344, 8345, 8351, 8366, 8378, 8382, 8390, 8391, 8393, 8396, 8397, 8402, 8409, 8412, 8422, 8425, 8429, 8431, 8432, 8438, 8441, 8444, 8454, 8462, 8465, 8470, 8484, 8492, 8494, 8501, 8504, 8507, 8508, 8509, 8512, 8516, 8517, 8526, 8533, 8538, 8540, 8544, 8546, 8549, 8554, 8562, 8568, 8574, 8575, 8577, 8590, 8601, 8604, 8605, 8609, 8611, 8616, 8619, 8625, 8627, 8629, 8631, 8649, 8652, 8657, 8668, 8669, 8671, 8688, 8691, 8697, 8705, 8709, 8717, 8726, 8735, 8745, 8746, 8749, 8754, 8763, 8768, 8775, 8782, 8783, 8787, 8793, 8795, 8796, 8799, 8805, 8806, 8807, 8815, 8819, 8828, 8838, 8840, 8841, 8848, 8853, 8859, 8869, 8887, 8888, 8895, 8901, 8903, 8917, 8918, 8924, 8925, 8927, 8935, 8947, 8953, 8960, 8964, 8968, 8969, 8971, 8977, 8979, 8982, 8985, 8986, 8988, 8993, 8995, 8996, 9006, 9007, 9010, 9020, 9021, 9031, 9036, 9040, 9043, 9045, 9049, 9053, 9056, 9068, 9072, 9077, 9078, 9080, 9085, 9087, 9099, 9104, 9106, 9109, 9131, 9134, 9142, 9145, 9152, 9157, 9158, 9167, 9171, 9178, 9180, 9191, 9193, 9194, 9197, 9198, 9199, 9215, 9223, 9224, 9230, 9239, 9247, 9256, 9260, 9263, 9266, 9267, 9270, 9271, 9273, 9284, 9285, 9287, 9289, 9290, 9292, 9303, 9304, 9305, 9311, 9315, 9322, 9332, 9335, 9337, 9343, 9345, 9352, 9354, 9367, 9370, 9399, 9411, 9416, 9431, 9432, 9438, 9440, 9443, 9444, 9446, 9452, 9453, 9457, 9459, 9460, 9468, 9477, 9482, 9496, 9501, 9516, 9526, 9539, 9544, 9547, 9563, 9568, 9575, 9578, 9579, 9583, 9595, 9599, 9607, 9613, 9615, 9616, 9618, 9621, 9637, 9642, 9656, 9657, 9660, 9670, 9671, 9673, 9676, 9684, 9685, 9686, 9689, 9691, 9693, 9698, 9699, 9704, 9710, 9711, 9717, 9721, 9730, 9735, 9740, 9742, 9745, 9748, 9749, 9752, 9758, 9759, 9760, 9762, 9763, 9764, 9773, 9775, 9780, 9781, 9783, 9792, 9793, 9801, 9802, 9813, 9816, 9821, 9823, 9843, 9851, 9865, 9867, 9870, 9871, 9881, 9883, 9888, 9892, 9898, 9900, 9929, 9933, 9942, 9944, 9952, 9956, 9958, 9960, 9964, 9971, 9972, 9973, 9974, 9975, 9979, 9983, 9986, 9987, 9989, 9992, 9993, 10003, 10004, 10006, 10010, 10011, 10013, 10014, 10019, 10021, 10022, 10024, 10026, 10028, 10035, 10040, 10049, 10053, 10054, 10060, 10064, 10065, 10069, 10073, 10074, 10077, 10084, 10085, 10091, 10097, 10102, 10104, 10116, 10118, 10120, 10129, 10131, 10132, 10133, 10146, 10158, 10159, 10160, 10161, 10167, 10170, 10173, 10175, 10177, 10178, 10183, 10184, 10185, 10187, 10195, 10206, 10211, 10226, 10231, 10233, 10237, 10242, 10248, 10268, 10270, 10275, 10280, 10281, 10283, 10284, 10285, 10289, 10293, 10297, 10300, 10302, 10305, 10311, 10320, 10321, 10324, 10326, 10328, 10331, 10334, 10341, 10346, 10347, 10348, 10355, 10367, 10369, 10370, 10373, 10397, 10398, 10403, 10409, 10417, 10422, 10424, 10425, 10432, 10446, 10452, 10455, 10458, 10465, 10466, 10485, 10514, 10533, 10540, 10546, 10547, 10559, 10561, 10565, 10567, 10574, 10576, 10577, 10581, 10591, 10596, 10601, 10602, 10603, 10611, 10615, 10616, 10623, 10624, 10626, 10629, 10630, 10640, 10650, 10657, 10662, 10664, 10689, 10690, 10702, 10705, 10712, 10713, 10714, 10717, 10720, 10722, 10730, 10737, 10746, 10749, 10755, 10759, 10760, 10761, 10763, 10770, 10778, 10779, 10783, 10785, 10790, 10791, 10799, 10815, 10828, 10834, 10838, 10841, 10852, 10857, 10861, 10864, 10865, 10866, 10867, 10868, 10869, 10871, 10875, 10881, 10883, 10886, 10889, 10892, 10903, 10906, 10909, 10910, 10917, 10921, 10927, 10930, 10938, 10941, 10942, 10943, 10947, 10955, 10957, 10965, 10976, 10977, 10983, 10985, 10988, 10989, 10994, 11017, 11027, 11028, 11031, 11036, 11042, 11046, 11055, 11064, 11066, 11067, 11074, 11079, 11080, 11086, 11087, 11089, 11095, 11096, 11100, 11105, 11114, 11117, 11122, 11131, 11132, 11134, 11135, 11136, 11139, 11147, 11150, 11151, 11155, 11157, 11158, 11159, 11180, 11181, 11198, 11199, 11201, 11202, 11204, 11206, 11217, 11223, 11225, 11231, 11233, 11234, 11236, 11239, 11246, 11253, 11256, 11259, 11264, 11280, 11282, 11284, 11291, 11310, 11311, 11312, 11314, 11323, 11327, 11329, 11332, 11333, 11337, 11339, 11350, 11358, 11360, 11364, 11372, 11375, 11384, 11405, 11406, 11418, 11424, 11434, 11438, 11454, 11458, 11462, 11473, 11476, 11478, 11479, 11486, 11495, 11499, 11503, 11507, 11523, 11527, 11529, 11531, 11533, 11535, 11537, 11545, 11551, 11559, 11563, 11567, 11587, 11590, 11592, 11593, 11596, 11598, 11599, 11603, 11608, 11621, 11624, 11626, 11634, 11635, 11647, 11650, 11652, 11654, 11659, 11660, 11661, 11663, 11666, 11668, 11672, 11675, 11679, 11680, 11683, 11685, 11688, 11693, 11694, 11700, 11709, 11726, 11749, 11753, 11758, 11762, 11765, 11766, 11779, 11790, 11794, 11799, 11804, 11806, 11808, 11814, 11817, 11820, 11821, 11839, 11840, 11843, 11850, 11861, 11862, 11863, 11877, 11884, 11887, 11893, 11897, 11900, 11908, 11910, 11911, 11916, 11925, 11933, 11935, 11945, 11953, 11957, 11959, 11963, 11972, 11974, 11976, 11977, 11978, 11979, 11986, 11993, 11996, 12003, 12006, 12008, 12015, 12018, 12022, 12027, 12035, 12042, 12054, 12056, 12063, 12064, 12069, 12077, 12081, 12095, 12104, 12106, 12108, 12111, 12113, 12125, 12126, 12129, 12132, 12140, 12142, 12143, 12151, 12152, 12157, 12161, 12180, 12189, 12196, 12199, 12207, 12208, 12223, 12225, 12248, 12253, 12254, 12258, 12261, 12267, 12269, 12280, 12281, 12282, 12286, 12303, 12304, 12307, 12311, 12325, 12344, 12350, 12354, 12355, 12356, 12364, 12366, 12379, 12382, 12383, 12388, 12390, 12391, 12395, 12398, 12400, 12403, 12407, 12408, 12410, 12416, 12418, 12419, 12427, 12428, 12432, 12437, 12440, 12441, 12451, 12462, 12464, 12467, 12469, 12472, 12473, 12480, 12485, 12487, 12489, 12496, 12499, 12500, 12505, 12512, 12517, 12536, 12537, 12543, 12546, 12549, 12561, 12569, 12570, 12572, 12573, 12575, 12576, 12577, 12586, 12590, 12594, 12608, 12610, 12611, 12613, 12616, 12617, 12629, 12632, 12637, 12646, 12661, 12664, 12671, 12672, 12680, 12689, 12691, 12696, 12707, 12710, 12715, 12716, 12721, 12724, 12728, 12729, 12734, 12738, 12744, 12747, 12749, 12750, 12756, 12759, 12766, 12767, 12770, 12780, 12781, 12782, 12787, 12804, 12807, 12810, 12821, 12823, 12826, 12827, 12844, 12847, 12857, 12859, 12861, 12887, 12895, 12896, 12897, 12902, 12906, 12912, 12916, 12919, 12920, 12922, 12928, 12940, 12941, 12956, 12957, 12962, 12963, 12968, 12971, 12976, 12979, 12985, 12989, 12995, 13000, 13007, 13023, 13031, 13034, 13045, 13051, 13056, 13057, 13060, 13068, 13070, 13074, 13075, 13076, 13077, 13079, 13081, 13084, 13093, 13096, 13108, 13121, 13129, 13131, 13135, 13136, 13141, 13144, 13150, 13151, 13152, 13157, 13159, 13161, 13165, 13167, 13171, 13176, 13184, 13187, 13190, 13198, 13200, 13202, 13214, 13219, 13222, 13229, 13235, 13237, 13239, 13241, 13254, 13260, 13267, 13270, 13273, 13275, 13278, 13287, 13289, 13304, 13305, 13315, 13326, 13332, 13339, 13355, 13365, 13366, 13371, 13379, 13387, 13392, 13393, 13395, 13396, 13409, 13422, 13423, 13424, 13430, 13433, 13435, 13436, 13442, 13448, 13457, 13463, 13466, 13470, 13473, 13476, 13482, 13490, 13492, 13494, 13507, 13516, 13525, 13533, 13544, 13546, 13549, 13554, 13562, 13566, 13569, 13578, 13580, 13581, 13599, 13604, 13607, 13618, 13625, 13626, 13632, 13636, 13637, 13650, 13653, 13656, 13660, 13664, 13666, 13674, 13676, 13683, 13685, 13689, 13696, 13701, 13702, 13704, 13709, 13730, 13732, 13735, 13742, 13746, 13748, 13749, 13753, 13756, 13757, 13758, 13769, 13774, 13782, 13783, 13788, 13790, 13794, 13805, 13811, 13828, 13834, 13842, 13846, 13848, 13849, 13851, 13853, 13856, 13859, 13862, 13866, 13869, 13870, 13874, 13877, 13879, 13884, 13885, 13887, 13897, 13901, 13903, 13904, 13905, 13915, 13916, 13926, 13936, 13940, 13942, 13944, 13945, 13951, 13954, 13976, 13990, 13999, 14005, 14011, 14013, 14023, 14030, 14043, 14049, 14052, 14063, 14069, 14071, 14073, 14080, 14089, 14090, 14093, 14102, 14106, 14108, 14110, 14120, 14126, 14137, 14141, 14142, 14144, 14155, 14161, 14163, 14170, 14176, 14181, 14184, 14189, 14190, 14197, 14202, 14203, 14205, 14218, 14225, 14226, 14235, 14237, 14243, 14255, 14259, 14272, 14273, 14277, 14285, 14290, 14297, 14315, 14317, 14321, 14334, 14343, 14344, 14353, 14363, 14370, 14373, 14378, 14383, 14387, 14390, 14392, 14395, 14402, 14404, 14425, 14429, 14433, 14440, 14448, 14453, 14459, 14465, 14467, 14474, 14478, 14479, 14482, 14484, 14488, 14490, 14492, 14504, 14505, 14511, 14512, 14517, 14526, 14540, 14544, 14546, 14549, 14550, 14556, 14561, 14564, 14570, 14575, 14606, 14611, 14614, 14616, 14620, 14622, 14625, 14626, 14638, 14640, 14642, 14643, 14653, 14657, 14665, 14678, 14681, 14682, 14687, 14688, 14689, 14696, 14707, 14712, 14715, 14716, 14720, 14728, 14736, 14753, 14754, 14755, 14760, 14768, 14773, 14775, 14776, 14786, 14787, 14793, 14796, 14803, 14804, 14805, 14814, 14824, 14826, 14827, 14838, 14844, 14846, 14849, 14854, 14858, 14861, 14864, 14866, 14872, 14879, 14882, 14887, 14889, 14896, 14901, 14903, 14908, 14914, 14923, 14926, 14927, 14936, 14940, 14948, 14955, 14957, 14958, 14960, 14968, 14971, 14977, 14984, 15007, 15010, 15011, 15015, 15022, 15023, 15042, 15044, 15049, 15050, 15057, 15058, 15059, 15062, 15068, 15072, 15078, 15080, 15087, 15089, 15091, 15095, 15105, 15107, 15111, 15116, 15128, 15131, 15132, 15136, 15143, 15145, 15152, 15160, 15165, 15167, 15170, 15175, 15181, 15184, 15186, 15187, 15194, 15199, 15201, 15237, 15243, 15246, 15251, 15259, 15261, 15263, 15266, 15267, 15271, 15275, 15294, 15319, 15321, 15329, 15339, 15345, 15352, 15360, 15366, 15395, 15399, 15403, 15409, 15412, 15427, 15430, 15435, 15437, 15438, 15439, 15442, 15452, 15455, 15456, 15457, 15458, 15466, 15468, 15474, 15477, 15482, 15487, 15492, 15493, 15500, 15509, 15514, 15517, 15521, 15527, 15535, 15545, 15564, 15566, 15579, 15582, 15592, 15595, 15599, 15606, 15607, 15616, 15618, 15619, 15625, 15639, 15640, 15643, 15649, 15650, 15651, 15659, 15661, 15668, 15670, 15678, 15686, 15691, 15700, 15702, 15705, 15709, 15710, 15716, 15719, 15727, 15730, 15732, 15734, 15743, 15747, 15755, 15760, 15762, 15763, 15764, 15765, 15766, 15770, 15785, 15787, 15790, 15800, 15802, 15809, 15817, 15829, 15830, 15831, 15842, 15847, 15850, 15853, 15858, 15867, 15868, 15872, 15873, 15882, 15893, 15910, 15911, 15916, 15923, 15926, 15932, 15934, 15954, 15956, 15957, 15960, 15963, 15990, 15992, 15993, 15996, 15998, 15999, 16006, 16013, 16014, 16017, 16019, 16021, 16022, 16030, 16034, 16044, 16045, 16050, 16051, 16055, 16056, 16057, 16071, 16075, 16083, 16084, 16088, 16092, 16098, 16108, 16112, 16117, 16125, 16131, 16136, 16139, 16142, 16144, 16145, 16152, 16158, 16165, 16167, 16169, 16192, 16196, 16201, 16203, 16204, 16210, 16212, 16214, 16217, 16218, 16227, 16232, 16247, 16254, 16255, 16262, 16273, 16296, 16301, 16312, 16316, 16317, 16323, 16327, 16329, 16335, 16336, 16345, 16347, 16357, 16360, 16361, 16367, 16369, 16371, 16375, 16379, 16387, 16392, 16399, 16411, 16416, 16418, 16420, 16421, 16425, 16428, 16431, 16433, 16438, 16439, 16440, 16443, 16451, 16457, 16477, 16480, 16482, 16483, 16487, 16489, 16492, 16497, 16500, 16503, 16505, 16508, 16511, 16513, 16518, 16520, 16524, 16530, 16534, 16536, 16541, 16542, 16558, 16559, 16560, 16565, 16570, 16574, 16579, 16583, 16618, 16623, 16625, 16627, 16632, 16634, 16639, 16641, 16644, 16651, 16662, 16663, 16667, 16669, 16670, 16675, 16680, 16685, 16686, 16687, 16690, 16692, 16700, 16708, 16713, 16717, 16720, 16724, 16732, 16737, 16738, 16741, 16744, 16745, 16756, 16758, 16760, 16763, 16777, 16780, 16787, 16794, 16796, 16797, 16798, 16801, 16806, 16807, 16810, 16812, 16816, 16817, 16828, 16832, 16839, 16843, 16844, 16851, 16858, 16867, 16870, 16881, 16888, 16891, 16893, 16896, 16902, 16906, 16911, 16921, 16923, 16925, 16936, 16939, 16942, 16945, 16948, 16949, 16950, 16951, 16956, 16973, 16975, 16978, 16991, 16993, 16998, 17002, 17003, 17006, 17012, 17013, 17036, 17040, 17059, 17066, 17074, 17084, 17088, 17093, 17100, 17112, 17115, 17120, 17124, 17126, 17132, 17135, 17139, 17145, 17152, 17157, 17158, 17163, 17168, 17170, 17177, 17183, 17186, 17189, 17193, 17195, 17200, 17202, 17215, 17226, 17232, 17241, 17251, 17252, 17255, 17262, 17265, 17277, 17278, 17287, 17289, 17293, 17301, 17304, 17315, 17319, 17320, 17322, 17325, 17326, 17327, 17351, 17352, 17353, 17370, 17371, 17377, 17384, 17390, 17393, 17405, 17408, 17414, 17420, 17426, 17428, 17434, 17437, 17445, 17460, 17461, 17468, 17472, 17473, 17482, 17484, 17486, 17489, 17493, 17494, 17497, 17506, 17508, 17509, 17514, 17517, 17523, 17524, 17525, 17529, 17541, 17545, 17549, 17552, 17561, 17562, 17567, 17568, 17569, 17575, 17577, 17579, 17584, 17589, 17593, 17594, 17595, 17596, 17599, 17605, 17608, 17615, 17618, 17622, 17629, 17637, 17638, 17646, 17647, 17650, 17657, 17664, 17667, 17682, 17694, 17695, 17698, 17699, 17703, 17710, 17711, 17713, 17716, 17717, 17722, 17723, 17734, 17738, 17740, 17755, 17756, 17758, 17767, 17770, 17776, 17789, 17791, 17801, 17810, 17826, 17829, 17836, 17837, 17843, 17845, 17847, 17852, 17854, 17855, 17862, 17866, 17867, 17868, 17871, 17875, 17877, 17879, 17880, 17897, 17900, 17903, 17908, 17911, 17917, 17925, 17927, 17928, 17931, 17932, 17934, 17936, 17937, 17956, 17957, 17967, 17972, 17980, 17981, 17984, 17989, 17998, 18002, 18003, 18004, 18019, 18020, 18024, 18027, 18029, 18039, 18044, 18045, 18049, 18051, 18066, 18076, 18087, 18092, 18109, 18123, 18127, 18141, 18142, 18153, 18158, 18173, 18177, 18190, 18191, 18193, 18198, 18207, 18212, 18214, 18217, 18218, 18221, 18230, 18232, 18246, 18247, 18249, 18258, 18269, 18270, 18271, 18274, 18275, 18283, 18285, 18286, 18306, 18330, 18332, 18339, 18340, 18346, 18364, 18368, 18375, 18376, 18378, 18382, 18403, 18405, 18413, 18418, 18421, 18422, 18433, 18434, 18441, 18446, 18447, 18449, 18450, 18452, 18453, 18461, 18468, 18475, 18478, 18480, 18492, 18504, 18507, 18536, 18537, 18539, 18544, 18546, 18547, 18554, 18556, 18558, 18566, 18569, 18575, 18586, 18605, 18607, 18609, 18613, 18614, 18618, 18620, 18623, 18632, 18634, 18640, 18642, 18644, 18649, 18653, 18654, 18662, 18667, 18671, 18679, 18683, 18692, 18702, 18703, 18725, 18732, 18734, 18737, 18738, 18740, 18747, 18751, 18753, 18764, 18765, 18769, 18771, 18774, 18779, 18780, 18784, 18794, 18796, 18799, 18815, 18819, 18820, 18824, 18825, 18827, 18831, 18833, 18835, 18839, 18841, 18846, 18865, 18872, 18873, 18878, 18883, 18890, 18895, 18898, 18912, 18923, 18929, 18935, 18937, 18944, 18947, 18953, 18971, 18972, 18973, 18979, 18981, 18982, 18984, 18985, 18986, 18988, 18989, 18990, 18994, 18996, 18999, 19001, 19008, 19016, 19022, 19028, 19029, 19030, 19035, 19037, 19040, 19042, 19046, 19053, 19056, 19059, 19060, 19069, 19071, 19087, 19088, 19090, 19091, 19099, 19104, 19108, 19109, 19110, 19111, 19114, 19118, 19134, 19136, 19151, 19155, 19156, 19159, 19197, 19198, 19206, 19226, 19227, 19230, 19231, 19238, 19239, 19242, 19251, 19256, 19258, 19270, 19271, 19282, 19286, 19289, 19291, 19292, 19301, 19305, 19307, 19309, 19311, 19321, 19331, 19332, 19335, 19343, 19347, 19355, 19373, 19374, 19375, 19382, 19384, 19390, 19401, 19405, 19406, 19412, 19419, 19440, 19441, 19447, 19451, 19452, 19460, 19461, 19464, 19473, 19475, 19485, 19488, 19492, 19493, 19507, 19511, 19512, 19520, 19521, 19523, 19525, 19526, 19532, 19534, 19543, 19546, 19561, 19566, 19567, 19569, 19571, 19577, 19578, 19579, 19580, 19587, 19588, 19590, 19596, 19600, 19605, 19606, 19607, 19610, 19611, 19613, 19622, 19624, 19632, 19635, 19636, 19643, 19644, 19650, 19656, 19662, 19669, 19679, 19692, 19694, 19695, 19706, 19708, 19709, 19711, 19721, 19731, 19735, 19737, 19743, 19748, 19755, 19763, 19764, 19767, 19769, 19772, 19776, 19779, 19788, 19792, 19793, 19794, 19810, 19812, 19813, 19827, 19831, 19837, 19850, 19852, 19853, 19865, 19866, 19869, 19872, 19874, 19875, 19879, 19894, 19899, 19900, 19906, 19912, 19913, 19920, 19931, 19937, 19938, 19941, 19944, 19950, 19952, 19953, 19955, 19957, 19958, 19960, 19977, 19978, 19980, 19984, 19987, 19989, 19994, 19999, 20003, 20007, 20008, 20012, 20018, 20021, 20026, 20030, 20036, 20044, 20046, 20049, 20080, 20082, 20084, 20091, 20093, 20095, 20098, 20099, 20100, 20109, 20110, 20112, 20118, 20119, 20125, 20127, 20132, 20134, 20135, 20136, 20139, 20140, 20141, 20143, 20144, 20147, 20158, 20169, 20179, 20181, 20183, 20191, 20192, 20193, 20203, 20207, 20210, 20215, 20217, 20218, 20220, 20230, 20232, 20242, 20246, 20248, 20251, 20265, 20272, 20287, 20299, 20300, 20301, 20304, 20305, 20307, 20320, 20328, 20332, 20345, 20350, 20359, 20369, 20373, 20378, 20381, 20389, 20401, 20403, 20409, 20423, 20445, 20447, 20455, 20460, 20461, 20468, 20472, 20483, 20485, 20492, 20495, 20496, 20504, 20507, 20510, 20512, 20518, 20528, 20529, 20540, 20541, 20559, 20575, 20576, 20578, 20580, 20585, 20589, 20595, 20604, 20613, 20615, 20616, 20617, 20618, 20620, 20624, 20634, 20638, 20646, 20653, 20662, 20668, 20673, 20677, 20680, 20683, 20692, 20694, 20705, 20706, 20708, 20712, 20714, 20716, 20720, 20736, 20737, 20738, 20742, 20743, 20757, 20758, 20761, 20764, 20777, 20781, 20782, 20796, 20800, 20802, 20813, 20823, 20824, 20827, 20829, 20835, 20837, 20853, 20858, 20866, 20869, 20884, 20888, 20890, 20894, 20899, 20902, 20906, 20908, 20909, 20912, 20914, 20917, 20924, 20929, 20931, 20936, 20941, 20949, 20951, 20956, 20958, 20960, 20962, 20964, 20965, 20966, 20977, 20984, 20991, 21006, 21010, 21012, 21015, 21021, 21024, 21030, 21032, 21033, 21035, 21040, 21044, 21047, 21048, 21050, 21056, 21059, 21064, 21068, 21072, 21073, 21087, 21093, 21097, 21098, 21101, 21107, 21111, 21121, 21126, 21131, 21136, 21143, 21154, 21156, 21158, 21180, 21182, 21184, 21186, 21191, 21195, 21196, 21200, 21204, 21232, 21240, 21241, 21242, 21252, 21254, 21270, 21274, 21284, 21288, 21290, 21291, 21298, 21307, 21309, 21311, 21315, 21317, 21321, 21327, 21330, 21332, 21340, 21341, 21345, 21348, 21351, 21354, 21355, 21359, 21362, 21365, 21372, 21375, 21376, 21379, 21381, 21384, 21397, 21408, 21409, 21410, 21412, 21414, 21415, 21419, 21423, 21431, 21435, 21444, 21445, 21446, 21451, 21456, 21460, 21469, 21481, 21485, 21489, 21490, 21493, 21497, 21500, 21501, 21508, 21509, 21518, 21523, 21532, 21534, 21538, 21544, 21547, 21554, 21555, 21558, 21561, 21562, 21563, 21564, 21571, 21573, 21577, 21581, 21582, 21590, 21597, 21605, 21609, 21617, 21618, 21621, 21627, 21631, 21639, 21641, 21644, 21649, 21651, 21652, 21653, 21660, 21661, 21662, 21669, 21671, 21683, 21687, 21691, 21699, 21701, 21705, 21707, 21708, 21710, 21724, 21725, 21732, 21743, 21749, 21751, 21755, 21764, 21766, 21771, 21776, 21780, 21781, 21785, 21786, 21800, 21804, 21805, 21814, 21827, 21828, 21840, 21844, 21850, 21852, 21853, 21856, 21865, 21869, 21875, 21876, 21881, 21884, 21891, 21903, 21904, 21910, 21914, 21917, 21918, 21921, 21926, 21927, 21929, 21932, 21933, 21934, 21941, 21943, 21957, 21958, 21959, 21960, 21961, 21970, 21971, 21972, 21973, 21975, 21976, 21978, 21984, 22000, 22002, 22010, 22011, 22012, 22014, 22018, 22029, 22045, 22046, 22047, 22052, 22053, 22056, 22063, 22065, 22066, 22071, 22074, 22081, 22082, 22086, 22088, 22095, 22098, 22100, 22101, 22102, 22103, 22109, 22118, 22124, 22126, 22128, 22133, 22143, 22148, 22152, 22153, 22161, 22168, 22175, 22179, 22182, 22184, 22191, 22195, 22196, 22203, 22205, 22209, 22211, 22216, 22221, 22222, 22223, 22227, 22228, 22229, 22235, 22242, 22245, 22247, 22250, 22253, 22254, 22263, 22268, 22271, 22272, 22281, 22285, 22292, 22304, 22309, 22311, 22312, 22316, 22317, 22318, 22319, 22321, 22322, 22324, 22326, 22329, 22330, 22333, 22335, 22337, 22341, 22342, 22350, 22354, 22355, 22364, 22367, 22368, 22372, 22379, 22385, 22389, 22390, 22391, 22417, 22426, 22428, 22429, 22430, 22437, 22439, 22444, 22445, 22452, 22454, 22455, 22458, 22462, 22463, 22465, 22466, 22467, 22475, 22479, 22481, 22482, 22486, 22490, 22506, 22510, 22511, 22512, 22518, 22520, 22521, 22532, 22534, 22536, 22538, 22559, 22564, 22567, 22569, 22572, 22585, 22593, 22595, 22599, 22600, 22603, 22605, 22611, 22615, 22618, 22625, 22637, 22638, 22644, 22655, 22675, 22681, 22683, 22702, 22705, 22711, 22714, 22717, 22720, 22751, 22756, 22768, 22783, 22786, 22799, 22800, 22806, 22809, 22815, 22819, 22820, 22821, 22829, 22834, 22835, 22842, 22849, 22859, 22885, 22896, 22910, 22912, 22914, 22916, 22919, 22920, 22921, 22931, 22937, 22938, 22951, 22953, 22965, 22991, 22995, 23000, 23007, 23008, 23009, 23013, 23030, 23032, 23033, 23038, 23040, 23042, 23051, 23059, 23060, 23062, 23064, 23066, 23071, 23076, 23083, 23084, 23087, 23088, 23094, 23096, 23106, 23110, 23118, 23120, 23134, 23137, 23145, 23147, 23156, 23157, 23158, 23160, 23161, 23164, 23173, 23178, 23185, 23209, 23221, 23229, 23232, 23233, 23236, 23240, 23241, 23244, 23247, 23256, 23260, 23264, 23268, 23276, 23282, 23292, 23293, 23294, 23304, 23306, 23309, 23315, 23319, 23321, 23324, 23333, 23342, 23344, 23346, 23347, 23356, 23357, 23359, 23360, 23365, 23373, 23379, 23392, 23406, 23409, 23422, 23426, 23428, 23430, 23444, 23447, 23449, 23456, 23457, 23459, 23464, 23469, 23472, 23480, 23491, 23496, 23497, 23499, 23501, 23502, 23505, 23536, 23544, 23549, 23550, 23553, 23557, 23570, 23577, 23587, 23600, 23603, 23607, 23615, 23619, 23620, 23622, 23626, 23636, 23638, 23639, 23649, 23654, 23657, 23658, 23662, 23666, 23667, 23673, 23676, 23677, 23679, 23688, 23689, 23691, 23710, 23714, 23715, 23719, 23721, 23723, 23731, 23733, 23735, 23738, 23740, 23743, 23745, 23746, 23748, 23749, 23753, 23759, 23775, 23776, 23779, 23783, 23784, 23787, 23793, 23815, 23819, 23821, 23823, 23831, 23833, 23834, 23835, 23841, 23842, 23844, 23845, 23850, 23860, 23862, 23863, 23865, 23866, 23869, 23878, 23882, 23889, 23893, 23894, 23895, 23898, 23914, 23917, 23920, 23921, 23923, 23931, 23933, 23934, 23939, 23940, 23943, 23952, 23959, 23976, 23983, 23992, 23994, 24001, 24013, 24014, 24016, 24018, 24022, 24023, 24040, 24044, 24050, 24058, 24063, 24068, 24071, 24074, 24087, 24091, 24092, 24093, 24095, 24096, 24104, 24105, 24113, 24116, 24121, 24131, 24133, 24134, 24142, 24158, 24165, 24170, 24175, 24176, 24178, 24179, 24189, 24196, 24203, 24206, 24210, 24213, 24217, 24218, 24237, 24240, 24247, 24254, 24263, 24266, 24267, 24268, 24270, 24282, 24285, 24292, 24295, 24297, 24298, 24299, 24300, 24301, 24302, 24309, 24311, 24317, 24318, 24333, 24335, 24338, 24343, 24346, 24353, 24356, 24361, 24363, 24378, 24392, 24396, 24404, 24409, 24411, 24412, 24426, 24429, 24431, 24432, 24433, 24436, 24440, 24445, 24447, 24448, 24450, 24451, 24458, 24463, 24465, 24467, 24468, 24475, 24477, 24479, 24481, 24484, 24486, 24487, 24502, 24504, 24510, 24512, 24515, 24536, 24537, 24550, 24554, 24555, 24561, 24563, 24565, 24570, 24571, 24577, 24587, 24593, 24603, 24615, 24616, 24623, 24632, 24633, 24640, 24641, 24643, 24650, 24653, 24660, 24676, 24677, 24679, 24692, 24700, 24702, 24713, 24714, 24719, 24726, 24727, 24735, 24738, 24742, 24743, 24744, 24753, 24760, 24771, 24773, 24774, 24778, 24781, 24782, 24784, 24791, 24792, 24801, 24821, 24830, 24838, 24845, 24853, 24854, 24856, 24857, 24859, 24863, 24869, 24870, 24883, 24892, 24895, 24896, 24897, 24900, 24904, 24907, 24908, 24914, 24918, 24919, 24925, 24927, 24931, 24934, 24935, 24944, 24947, 24958, 24959, 24963, 24984, 24986, 24998, 25000, 25012, 25032, 25033, 25038, 25039, 25045, 25048, 25057, 25059, 25062, 25065, 25066, 25071, 25072, 25085, 25091, 25098, 25100, 25102, 25107, 25115, 25118, 25130, 25136, 25139, 25143, 25163, 25167, 25179, 25185, 25186, 25191, 25192, 25200, 25210, 25217, 25218, 25222, 25231, 25240, 25246, 25249, 25250, 25258, 25260, 25264, 25265, 25274, 25278, 25281, 25286, 25289, 25300, 25302, 25303, 25304, 25317, 25329, 25333, 25338, 25342, 25348, 25381, 25382, 25386, 25390, 25394, 25398, 25413, 25425, 25433, 25437, 25439, 25442, 25443, 25452, 25454, 25455, 25471, 25475, 25484, 25486, 25496, 25497, 25499, 25502, 25509, 25510, 25517, 25520, 25523, 25526, 25527, 25533, 25535, 25539, 25552, 25558, 25561, 25565, 25569, 25571, 25575, 25591, 25611, 25616, 25619, 25620, 25622, 25631, 25636, 25637, 25639, 25645, 25649, 25652, 25654, 25657, 25658, 25661, 25664, 25665, 25666, 25668, 25673, 25680, 25698, 25700, 25701, 25712, 25713, 25720, 25734, 25735, 25737, 25740, 25741, 25742, 25743, 25750, 25755, 25773, 25783, 25785, 25793, 25798, 25809, 25814, 25832, 25835, 25848, 25849, 25851, 25852, 25864, 25866, 25872, 25873, 25878, 25880, 25888, 25899, 25912, 25915, 25922, 25923, 25925, 25926, 25928, 25931, 25934, 25941, 25945, 25953, 25960, 25967, 25978, 25980, 25997, 26007, 26013, 26042, 26046, 26055, 26056, 26065, 26069, 26082, 26086, 26088, 26090, 26100, 26110, 26111, 26116, 26118, 26124, 26138, 26148, 26152, 26153, 26165, 26173, 26183, 26184, 26192, 26195, 26208, 26209, 26210, 26211, 26218, 26225, 26226, 26227, 26238, 26248, 26250, 26251, 26254, 26257, 26263, 26268, 26271, 26273, 26275, 26276, 26287, 26288, 26294, 26295, 26297, 26303, 26307, 26312, 26315, 26317, 26319, 26323, 26325, 26327, 26350, 26353, 26358, 26360, 26366, 26371, 26387, 26389, 26390, 26398, 26402, 26403, 26406, 26425, 26430, 26431, 26433, 26434, 26438, 26443, 26447, 26449, 26450, 26452, 26456, 26464, 26466, 26478, 26479, 26482, 26485, 26488, 26494, 26499, 26508, 26528, 26539, 26550, 26554, 26556, 26558, 26559, 26566, 26572, 26574, 26579, 26584, 26590, 26597, 26601, 26605, 26609, 26612, 26614, 26616, 26617, 26623, 26627, 26629, 26642, 26653, 26656, 26671, 26678, 26679, 26680, 26682, 26683, 26684, 26685, 26687, 26690, 26694, 26704, 26715, 26716, 26719, 26730, 26733, 26737, 26743, 26745, 26746, 26748, 26753, 26758, 26767, 26779, 26791, 26805, 26806, 26807, 26811, 26813, 26814, 26818, 26820, 26825, 26829, 26835, 26843, 26849, 26851, 26853, 26855, 26856, 26859, 26863, 26865, 26868, 26879, 26885, 26890, 26891, 26897, 26908, 26915, 26917, 26933, 26935, 26936, 26937, 26942, 26946, 26954, 26964, 26967, 26972, 26978, 26988, 26991, 26997, 27011, 27013, 27021, 27030, 27033, 27039, 27043, 27045, 27057, 27059, 27063, 27065, 27066, 27068, 27070, 27072, 27083, 27088, 27092, 27107, 27108, 27109, 27113, 27115, 27124, 27126, 27132, 27139, 27140, 27154, 27162, 27163, 27168, 27184, 27190, 27193, 27203, 27212, 27217, 27226, 27232, 27234, 27240, 27242, 27244, 27249, 27250, 27259, 27261, 27262, 27267, 27273, 27274, 27275, 27282, 27289, 27291, 27293, 27295, 27296, 27302, 27303, 27304, 27316, 27317, 27318, 27322, 27328, 27330, 27337, 27343, 27347, 27361, 27362, 27365, 27367, 27369, 27379, 27396, 27398, 27400, 27401, 27402, 27406, 27407, 27410, 27413, 27415, 27424, 27428, 27429, 27430, 27435, 27436, 27438, 27462, 27505, 27509, 27518, 27519, 27522, 27528, 27529, 27532, 27534, 27535, 27539, 27541, 27542, 27547, 27548, 27549, 27550, 27556, 27558, 27575, 27583, 27589, 27591, 27593, 27597, 27602, 27608, 27609, 27615, 27617, 27621, 27623, 27624, 27628, 27633, 27638, 27642, 27644, 27647, 27649, 27655, 27656, 27659, 27672, 27674, 27676, 27678, 27681, 27682, 27685, 27689, 27695, 27703, 27706, 27709, 27714, 27718, 27732, 27733, 27734, 27735, 27737, 27739, 27745, 27750, 27753, 27760, 27762, 27780, 27781, 27783, 27785, 27788, 27803, 27807, 27809, 27817, 27820, 27821, 27840, 27842, 27844, 27845, 27847, 27850, 27857, 27858, 27867, 27875, 27885, 27891, 27908, 27916, 27922, 27923, 27925, 27926, 27943, 27947, 27948, 27949, 27959, 27964, 27965, 27966, 27972, 27976, 27988, 27992, 27993, 27994, 27995, 28000, 28007, 28009, 28013, 28018, 28020, 28023, 28024, 28027, 28032, 28037, 28043, 28046, 28050, 28053, 28060, 28061, 28064, 28083, 28084, 28085, 28087, 28091, 28092, 28096, 28102, 28106, 28117, 28118, 28121, 28124, 28130, 28134, 28136, 28140, 28149, 28157, 28158, 28159, 28163, 28166, 28170, 28172, 28173, 28177, 28181, 28182, 28191, 28196, 28210, 28215, 28218, 28235, 28240, 28245, 28259, 28264, 28265, 28269, 28275, 28285, 28287, 28289, 28292, 28293, 28297, 28301, 28306, 28316, 28327, 28335, 28336, 28339, 28342, 28347, 28348, 28350, 28352, 28358, 28359, 28369, 28376, 28382, 28388, 28392, 28394, 28398, 28401, 28403, 28404, 28410, 28418, 28421, 28425, 28427, 28428, 28429, 28433, 28441, 28448, 28450, 28456, 28463, 28466, 28470, 28471, 28472, 28478, 28481, 28486, 28493, 28500, 28507, 28516, 28521, 28523, 28529, 28530, 28534, 28544, 28558, 28559, 28561, 28569, 28587, 28596, 28597, 28607, 28613, 28616, 28621, 28624, 28627, 28632, 28640, 28648, 28650, 28653, 28661, 28665, 28669, 28678, 28682, 28686, 28690, 28696, 28699, 28710, 28726, 28728, 28735, 28748, 28752, 28757, 28760, 28766, 28771, 28772, 28783, 28805, 28816, 28819, 28824, 28825, 28829, 28831, 28835, 28836, 28838, 28840, 28841, 28842, 28855, 28857, 28858, 28861, 28863, 28865, 28867, 28878, 28881, 28883, 28885, 28886, 28891, 28893, 28895, 28899, 28903, 28907, 28910, 28915, 28921, 28923, 28927, 28937, 28942, 28944, 28954, 28963, 28964, 28967, 28970, 28973, 28976, 28978, 28982, 29013, 29014, 29024, 29028, 29032, 29034, 29039, 29044, 29049, 29050, 29056, 29057, 29059, 29060, 29067, 29069, 29070, 29078, 29080, 29081, 29082, 29083, 29087, 29091, 29095, 29099, 29109, 29119, 29121, 29131, 29135, 29137, 29140, 29147, 29149, 29154, 29156, 29163, 29166, 29174, 29175, 29177, 29181, 29187, 29189, 29203, 29205, 29208, 29211, 29213, 29216, 29220, 29221, 29222, 29225, 29227, 29231, 29242, 29244, 29248, 29254, 29269, 29271, 29273, 29276, 29287, 29289, 29290, 29302, 29312, 29314, 29320, 29321, 29326, 29334, 29336, 29338, 29352, 29357, 29362, 29366, 29370, 29371, 29375, 29381, 29387, 29388, 29392, 29393, 29405, 29406, 29407, 29408, 29411, 29418, 29420, 29422, 29424, 29438, 29440, 29445, 29454, 29459, 29463, 29464, 29477, 29484, 29487, 29488, 29492, 29493, 29494, 29502, 29503, 29507, 29508, 29511, 29522, 29527, 29532, 29534, 29536, 29545, 29549, 29550, 29565, 29579, 29601, 29603, 29606, 29611, 29614, 29618, 29619, 29620, 29621, 29625, 29627, 29629, 29631, 29645, 29655, 29662, 29669, 29671, 29672, 29673, 29680, 29683, 29693, 29702, 29709, 29716, 29723, 29735, 29739, 29740, 29742, 29745, 29746, 29754, 29756, 29758, 29781, 29784, 29809, 29810, 29815, 29828, 29837, 29838, 29839, 29843, 29844, 29846, 29848, 29854, 29856, 29857, 29874, 29880, 29882, 29892, 29896, 29908, 29913, 29921, 29934, 29938, 29940, 29944, 29945, 29949, 29955, 29956, 29960, 29973, 29974, 29976, 29978, 29979, 29984, 29985, 29992, 29994, 29999, 30000, 30007, 30022, 30023, 30033, 30038, 30041, 30043, 30049, 30054, 30061, 30062, 30066, 30070, 30073, 30077, 30079, 30080, 30082, 30092, 30093, 30094, 30100, 30102, 30103, 30108, 30115, 30127, 30129, 30131, 30133, 30135, 30142, 30150, 30152, 30156, 30159, 30169, 30170, 30175, 30182, 30192, 30198, 30199, 30206, 30230, 30239, 30245, 30248, 30252, 30255, 30261, 30263, 30272, 30273, 30281, 30284, 30289, 30290, 30296, 30303, 30314, 30325, 30337, 30339, 30343, 30345, 30348, 30354, 30356, 30358, 30366, 30375, 30377, 30386, 30391, 30401, 30405, 30407, 30410, 30411, 30412, 30420, 30427, 30446, 30447, 30453, 30459, 30468, 30477, 30483, 30495, 30500, 30501, 30506, 30511, 30521, 30528, 30530, 30531, 30533, 30535, 30541, 30545, 30546, 30551, 30554, 30571, 30577, 30584, 30586, 30587, 30589, 30594, 30597, 30599, 30601, 30602, 30605, 30610, 30614, 30615, 30625, 30626, 30627, 30630, 30631, 30632, 30651, 30665, 30669, 30675, 30679, 30688, 30695, 30696, 30704, 30705, 30709, 30710, 30715, 30716, 30717, 30719, 30727, 30733, 30738, 30742, 30747, 30754, 30756, 30760, 30762, 30764, 30766, 30767, 30768, 30781, 30795, 30796, 30801, 30804, 30806, 30812, 30816, 30818, 30820, 30821, 30823, 30824, 30829, 30832, 30834, 30835, 30837, 30847, 30850, 30855, 30856, 30861, 30868, 30877, 30878, 30889, 30892, 30893, 30903, 30904, 30911, 30929, 30933, 30935, 30940, 30941, 30944, 30947, 30953, 30956, 30958, 30959, 30961, 30964, 30969, 30972, 30975, 30981, 30997, 31001, 31006, 31012, 31015, 31017, 31018, 31020, 31023, 31026, 31035, 31036, 31038, 31051, 31055, 31059, 31060, 31071, 31080, 31088, 31089, 31091, 31094, 31096, 31097, 31100, 31109, 31119, 31157, 31160, 31164, 31167, 31171, 31172, 31176, 31182, 31186, 31193, 31200, 31207, 31209, 31216, 31223, 31225, 31228, 31232, 31233, 31235, 31245, 31253, 31256, 31265, 31267, 31268, 31276]

In [14]:
test_dataset = Dataset_Generator(h5_file, test_indx, reshape_size=64,
                                     means=statistics["mean"].div_(550),
                                     stds=statistics["std"].div_(550), only_channels=only_channels,
                                     only_classes=only_classes)
testloader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=num_workers)

In [15]:
shuffle_times = 5

In [16]:
def shuffle_pixels_in_channel(channel, image):
    im = copy.deepcopy(image)
    channel_shape = im[channel].shape
    #breakpoint()
    #im[0].flatten()[torch.randperm(len(im[0].flatten()))]
    #arr = np.asarray(im[channel].flatten())
    #np.random.shuffle(arr)
    plt.imshow(im[channel].cpu())
    im[channel] = im[channel].flatten()[torch.randperm(len(im[channel].flatten()))].reshape(channel_shape)
    plt.imshow(im[channel].cpu())
    return im

In [19]:
import copy
correct = 0.
total = 0.
y_true = list()
y_pred = list()
y_pred_per_channel = {}

for n in range(num_channels):
    y_pred_per_channel["y_pred_{}".format(n)] = list()

#total_length = len(testloader)
#iteration = 1
with torch.no_grad():
    for data in testloader:
        indx = (data["object_number"] != -1).reshape(-1)
        if indx.sum() > 0:
            inputs, labels = data["image"][indx], data["label"][indx]

            inputs, labels = inputs.to(device), labels.to(device)     
            inputs = inputs.float()
            labels = labels.reshape(-1)

            outputs = model(inputs)
            pred = outputs.argmax(dim=1)
            _ , predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (labels.reshape(-1) == predicted).sum().item()
            for i in range(len(pred)):
                y_true.append(labels[i].item())
                y_pred.append(pred[i].item())
        ### Predited labels of shuffled data
        #print("Finish {} %".format((iteration / total_length)*100))
        #iteration += 1
        #torch.cuda.empty_cache()

print('Accuracy of the network on the %d test images: %d %%' % (len(test_dataset),
        100 * correct / total))
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
f1_score_original = f1_score(y_true, y_pred, average=None, labels=np.arange(num_classes))
df = pd.DataFrame(np.atleast_2d(f1_score_original), columns=class_names)
print(df.to_string())

Accuracy of the network on the 6256 test images: 72 %
                   precision    recall  f1-score   support

          unknown     0.1888    0.3457    0.2442       243
           CD4+ T     0.7703    0.9861    0.8649       932
           CD8+ T     0.5522    0.8342    0.6645       374
 CD15+ neutrophil     1.0000    0.7755    0.8735      3603
   CD14+ monocyte     0.4983    0.5070    0.5026       286
          CD19+ B     0.3822    0.3488    0.3647       172
         CD56+ NK     0.4815    0.6364    0.5482       143
              NKT     0.3316    0.3077    0.3192       208
       eosinophil     0.2145    0.3119    0.2541       295

         accuracy                         0.7291      6256
        macro avg     0.4910    0.5615    0.5151      6256
     weighted avg     0.7965    0.7291    0.7493      6256

    unknown    CD4+ T    CD8+ T   CD15+ neutrophil   CD14+ monocyte   CD19+ B   CD56+ NK       NKT   eosinophil
0  0.244186  0.864941  0.664537           0.873534           0.5

In [23]:
for channel in range(inputs[0].shape[0]):
                pred_images = []
                for image in inputs:
                    image_shuffled_times = []
                    for t in range(shuffle_times):
                        im = shuffle_pixels_in_channel(channel, image)
                        image_shuffled_times.append(im)
                    image_shuffled_times = torch.stack(image_shuffled_times, dim=0)
                    pred_image_shuffled_times = model(image_shuffled_times).argmax(dim=1)
                    pred_images.append(pred_image_shuffled_times)
                for i_ch in range(len(pred_images)):
                    y_pred_per_channel["y_pred_{}".format(channel)].append(pred_images[i_ch])

KeyboardInterrupt: 

In [ ]:
f1_scores_per_channel = {}
for channel, y_pred_per_ch in y_pred_per_channel.items():
    f1_scores_per_shuffle = []
    y_pred_per_im = torch.stack(y_pred_per_ch, dim=0).T
    for y_pred_per_shuffle in y_pred_per_im:
        f1_scores_per_shuffle.append(f1_score(y_true, y_pred_per_shuffle.cpu(), average=None, labels=np.arange(num_classes)))
    f1_score_all_per_channel = np.stack(f1_scores_per_shuffle)

    f1_diff_from_original = f1_score_original - f1_score_all_per_channel
    df_diff = pd.DataFrame(np.atleast_2d(f1_diff_from_original), columns=opt.class_names)
    # df_diff = df_diff[['G1', 'G2', 'S', 'Prophase', 'Metaphase', 'Anaphase', 'Telophase']]
    fig = plt.figure(figsize=(10, 5))
    bp = df_diff.boxplot()
    fig.savefig(os.path.join(opt.path_to_save_results, "wbc-shuffle_method-model-{}-channel-{}.png".format(str(os.path.basename(os.path.normpath(opt.path_to_model))), str(channel))))